<center><h1>4 DOF Time Domain Model Notebook</h1></center>
<br>
<center><h3>Notebook written by <em>Sreenath S</em></h3></center>

This notebook is an attempt at proper documentation of the development of the 4 DOF Time domain model for **Ship Manoeuvring in Waves**

<p><strong>Jupyter Notebook Shortcuts</strong></p>
<ul>
<li>Toggle between edit and command mode with <code>Esc</code> and <code>Enter</code>, respectively.</li>
<li>Once in command mode:
<ul>
<li>Scroll up and down your cells with your <code>Up</code> and <code>Down</code> keys.</li>
<li>Press <code>A</code> or <code>B</code> to insert a new cell above or below the active cell.</li>
<li><code>M</code> will transform the active cell to a Markdown cell.</li>
<li><code>Y</code> will set the active cell to a code cell.</li>
<li><code>D + D</code> (<code>D</code> twice) will delete the active cell.</li>
<li><code>Z</code> will undo cell deletion.</li>
<li>Hold <code>Shift</code> and press <code>Up</code> or <code>Down</code> to select multiple cells at once.
<ul>
<li>With multiple cells selected, <code>Shift + M</code> will merge your selection.</li>
</ul>
</li>
</ul>
</li>
<li><code>Ctrl + Shift + -</code>, in edit mode, will split the active cell at the cursor.</li>
<li>You can also click and <code>Shift + Click</code> in the margin to the left of your cells to select them.</li>
</ul>

**STEP 1) Import the necessary modules**


In [42]:
# Standard Library Imports

import pandas as pd
import numpy  as np
import math
from scipy import constants
import matplotlib.pyplot as plt

# Custom Library Imports

from modules import math_operations


**STEP 2) Read the necessary Input Data**

In [43]:
############################# Intialization ###############################

# 1. Universal Constant

g = constants.g                                       # Acceleration due to gravity
rho = 1025                                            # Density of sea water
pi = constants.pi                                     # Pi

# 2. Hull Properties

df = pd.read_csv('data/vessel_parameters.csv', index_col='parameter', comment='#')

M = df['value']['M']                                  # Ship Mass kg
                         
I_B = np.array([ [df['value']['I_X'],0,0],            # Moment and Product of inertia Matrix
                 [0,df['value']['I_Y'],0],
                 [2,0,df['value']['I_Z']] ])

V_disp = df['value']['V_disp']                        # Volume Displacement

LPP = df['value']['LPP']                              # Length between perpendiculars

LCB = df['value']['LCB']                              # Location of LCB from body-fixed CS

D = df['value']['D']                                  # Breadth of ship

B = df['value']['B']                                  # Breadth of ship

T = df['value']['T']                                  # Draft of ship

GM_T = df['value']['GM_T']                            # Transverse metacentric height

GM_L = df['value']['GM_L']                            # Longitudinal metacentric height

R_G = np.array([ [df['value']['X_G']],                # Vector of C.O.G
                 [df['value']['Y_G']], 
                 [df['value']['Z_G']] ])

Yv_dash = df['value']["Yv'"]                          # First order hydrodynamic derivatives
YR_dash = df['value']["YR'"]                          # 
Nv_dash = df['value']["Nv'"]                          #
NR_dash = df['value']["NR'"]                          #

Xvr_dash = df['value']["Xvr'"]                        # Second order hydrodynamic derivatives
Xvv_dash = df['value']["Xvv'"]                        # 
Xrr_dash = df['value']["Xrr'"]                        #

Yvvr_dash = df['value']["Yvvr'"]                      # Third order hydrodynamic derivatives 
Yvrr_dash = df['value']["Yvrr'"]                      #
Nvvr_dash = df['value']["Nvvr'"]                      #
Nvrr_dash = df['value']["Nvrr'"]                      # 
Yvvv_dash = df['value']["Yvvv'"]                      #
Yrrr_dash = df['value']["Yrrr'"]                      #
Nvvv_dash = df['value']["Nvvv'"]                      #
Nrrr_dash = df['value']["Nrrr'"]                      #

Xvvvv_dash = df['value']["Xvvvv'"]                    # Fourth order hydrodynamic derivative

# 3. Propeller Properties

Nprop = df['value']["Nprop"]                          # No. of Propellers
Pd    = df['value']["Pd"]                             # Propeller diameter
w     = df['value']["w"]                              # Wake Fraction
t     = df['value']["t"]                              # Thrust Deduction Fraction

# 4. Experimental Conditions

Fn = df['value']["Fn"]                                # Froude Number

# 5. Rt vs Vs Data

df2 = pd.read_csv('data/Rt__vs__Vs.csv',  comment='#')
Vs_points  = df2['Vs'].to_numpy()
Rt_points  = df2['Rt'].to_numpy()

# 6. Read Kt vs J Data

df3 = pd.read_csv('data/Kt__vs__J.csv',  comment='#')
J_points  = df3['J'].to_numpy()
Kt_points = df3['Kt'].to_numpy()

In [44]:
############################# Preparatory Calculations ###############################
#                                                                                    #
#     This section includes some basic set of calculations which must be             #
#     carried out prior to the main simulation.                                      #
#                                                                                    #
######################################################################################

print("Simulation started")

# 1. Prediction of Self Propulsion RPS from Froude number

Vs = Fn * math.sqrt(g*LPP)

Nmin = 1                                              # Define a minimum and maximum value for Propeller RPM
Nmax = 50

while True:

    N = (Nmin+Nmax)/2

    J = (Vs*(1-w))/(N*Pd)

    Kt = math_operations.cspline_interp(J,J_points, Kt_points)

    Thrust = Nprop * Kt * rho * N**2 * Pd**4

    Rt = math_operations.cspline_interp(Vs,Vs_points, Rt_points)

    var = Rt-(Thrust*(1-t))

    if  ( round(var,4) == 0 ):
        break
    elif ( var > 0 ):  
        Nmin = N
    elif ( var < 0 ):
        Nmax = N

print("Still water propeller RPM =", N)

# 2. Calculation of the still water equilibrium with action of constant wind.


print("Simulation Ended")


Simulation started
Still water propeller RPM = 3.6847103475265612
Simulation Ended
